# glm4 stream调用

In [1]:
import os
import torch
from threading import Thread
from transformers import AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer, AutoModel

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
MODEL_PATH = '/opt/Data/ModelWeight/THUDM/glm-4-9b-chat'
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    encode_special_tokens=True
)
model = AutoModel.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    device_map="cuda").eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

history = []
max_length = 8192
top_p = 0.8
temperature = 0.6
stop = StopOnTokens()

user_input = "如何看待小孩子每天都喜欢玩，不太爱学习"
history.append([user_input, ""])
messages = []

for idx, (user_msg, model_msg) in enumerate(history):
    if idx == len(history) - 1 and not model_msg:
        messages.append({"role": "user", "content": user_msg})
        break
    if user_msg:
        messages.append({"role": "user", "content": user_msg})
    if model_msg:
        messages.append({"role": "assistant", "content": model_msg})

model_inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt"
).to(model.device)

streamer = TextIteratorStreamer(
    tokenizer=tokenizer,
    timeout=60,
    skip_prompt=True,
    skip_special_tokens=True
)

generate_kwargs = {
    "input_ids": model_inputs,
    "streamer": streamer,
    "max_new_tokens": max_length,
    "do_sample": True,
    "top_p": top_p,
    "temperature": temperature,
    "stopping_criteria": StoppingCriteriaList([stop]),
    "repetition_penalty": 1.2,
    "eos_token_id": model.config.eos_token_id,
}

t = Thread(target=model.generate, kwargs=generate_kwargs)
t.start()

print("GLM-4:", end="", flush=True)

for new_token in streamer:
    if new_token:
        print(new_token, end="", flush=True)
        history[-1][1] += new_token

history[-1][1] = history[-1][1].strip()

GLM-4:
孩子每天喜欢玩耍而不太热衷于学习的现象在儿童心理学中是正常且普遍的。以下是对这一现象的一些看法：

1. **生理和心理发展特点**：
   - 儿童时期是人类大脑发育最快的阶段之一，孩子们在这个时期的注意力、自控力和认知能力还在发展中。
   - 玩耍对于孩子的身心健康发展至关重要，它有助于他们探索世界、培养社交技能和解决问题的能力。

2. **教育理念**：
   - 当前的教育观念越来越强调“快乐学习”，即通过游戏化的方式让孩子在学习中获得乐趣，从而提高他们的学习兴趣和学习效果。
   - 强调素质教育而非单纯的应试教育，鼓励孩子在德智体美劳各方面全面发展。

3. **家庭教育与引导**：
   - 家长可以通过创造良好的家庭环境来激发孩子的学习兴趣，比如提供丰富的图书资源、科学实验器材等。
   - 通过亲子活动或参与社会实践等方式，将学习内容融入日常生活，使孩子在实际操作中发现知识的重要性。

4. **社会因素**：
   - 在竞争激烈的社会环境中，家长可能会对孩子有较高的期望值，这可能导致孩子感到压力重重，不愿意主动去学习。
   - 学校的教育方式和评价体系也可能影响学生的学习积极性。

5. **平衡玩耍与学习**：
   - 作为家长和教育者，应当找到玩耍与学习之间的平衡点，既不剥夺孩子应有的童年乐趣，也不忽视其学业成长。
   - 鼓励孩子进行有益身心的户外活动和集体运动，同时合理安排课业负担。

总之，对待孩子喜欢玩耍而较少热爱学习的情况，应该采取理解和支持的态度，结合科学的育儿方法，帮助孩子健康成长。在这个过程中，家长的耐心和理解尤为重要。

In [2]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

history = []
def runChat(user_input:str):
    history.append([user_input, ""])
    messages = []
    
    for idx, (user_msg, model_msg) in enumerate(history):
        if idx == len(history) - 1 and not model_msg:
            messages.append({"role": "user", "content": user_msg})
            break
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if model_msg:
            messages.append({"role": "assistant", "content": model_msg})
    
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)
    
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer,
        timeout=60,
        skip_prompt=True,
        skip_special_tokens=True
    )
    
    generate_kwargs = {
        "input_ids": model_inputs,
        "streamer": streamer,
        "max_new_tokens": 8192,
        "do_sample": True,
        "top_p": 0.8,
        "temperature":  0.6,
        "stopping_criteria": StoppingCriteriaList([StopOnTokens()]),
        "repetition_penalty": 1.2,
        "eos_token_id": model.config.eos_token_id,
    }
    
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    
    print("GLM-4:", end="", flush=True)

    generatetxt=""
    for new_token in streamer:
        if new_token:
            print(new_token, end="", flush=True)
            history[-1][1] += new_token
            generatetxt+=new_token
    t.join()
    return generatetxt

user_input = "作为一名软件工程师，如何提高工作效率"
res = runChat(user_input)
print(res)
history[-1][1] = history[-1][1].strip()

GLM-4:
作为软件工程师，提升工作效率是一个持续的过程。以下是一些建议帮助您提高个人工作效率：

1. **时间管理**：
   - 使用番茄工作法等技巧来集中注意力。
   - 制定清晰的日程表和任务清单。

2. **优先级排序**：
   - 学会区分紧急与重要的事务，并按照重要性进行排序处理。

3. **技术工具**：
   - 利用版本控制、项目管理工具（如Jira）、代码审查工具等自动化流程。
   - 学习使用快捷键和脚本来自动化重复性任务。

4. **编码习惯**：
   - 保持良好的编程规范和注释，使代码易于阅读和维护。
   - 遵循设计模式和最佳实践。

5. **团队协作**：
   - 与团队成员保持沟通畅通，及时分享进度和问题。
   - 参加代码评审会议，从他人反馈中学习。

6. **知识积累**：
   - 持续学习和更新自己的技能库。
   - 关注行业动态和技术趋势。

7. **避免干扰**：
   - 在需要专注工作时关闭不必要的通知和社交媒体应用。
   - 创建一个有利于工作的环境。

8. **健康生活**：
   - 保证充足的睡眠和适当的运动。
   - 注意饮食均衡，以维持身体和精神状态良好。

9. **反思总结**：
   - 定期回顾过去的工作表现，找出可以改进的地方。
   - 从失败的项目或错误中学到经验教训。

10. **合理分工**：
    - 了解自己擅长和不擅长的领域，将任务分配给最合适的人选。

11. **减少等待时间**：
    - 优化开发过程，缩短编译、测试和部署的时间。

12. **心理调适**：
    - 学会放松心情，适时调整心态。
    - 当遇到压力时，寻求同事或专业人士的帮助。

通过以上方法，您可以逐步提高工作效率，从而更好地完成工作任务。记住，每个人的情况不同，找到适合自己的方法是关键。
作为软件工程师，提升工作效率是一个持续的过程。以下是一些建议帮助您提高个人工作效率：

1. **时间管理**：
   - 使用番茄工作法等技巧来集中注意力。
   - 制定清晰的日程表和任务清单。

2. **优先级排序**：
   - 学会区分紧急与重要的事务，并按照重要性进行排序处理。

3. **技术工具**：
   - 利用版本控制、项目管理工具（如Ji

In [18]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

history = []
def runChat(user_input:str):
    history.append([user_input, ""])
    messages = []
    
    for idx, (user_msg, model_msg) in enumerate(history):
        if idx == len(history) - 1 and not model_msg:
            messages.append({"role": "user", "content": user_msg})
            break
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if model_msg:
            messages.append({"role": "assistant", "content": model_msg})
    
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)
    
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer,
        timeout=60,
        skip_prompt=True,
        skip_special_tokens=True
    )
    
    generate_kwargs = {
        "input_ids": model_inputs,
        "streamer": streamer,
        "max_new_tokens": 8192,
        "do_sample": True,
        "top_p": 0.8,
        "temperature":  0.6,
        "stopping_criteria": StoppingCriteriaList([StopOnTokens()]),
        "repetition_penalty": 1.2,
        "eos_token_id": model.config.eos_token_id,
    }
    
    # t = Thread(target=model.generate, kwargs=generate_kwargs)
    # t.start()

    result = model.generate(**generate_kwargs)
    print(f"result:{result}")
    print("GLM-4:", end="", flush=True)

    generatetxt=""
    for new_token in streamer:
        if new_token:
            print(new_token, end="", flush=True)
            history[-1][1] += new_token
            generatetxt+=new_token
#     t.join()
#     return generatetxt

user_input = "你好，介绍一下南京"
res = runChat(user_input)
print(res)
# history[-1][1] = history[-1][1].strip()

result:tensor([[151331, 151333, 151336,    198, 109377,   3837, 117392, 101494, 151337,
            198, 109377,   6313, 101494, 103448, 106486, 101119, 111069, 122231,
           3837, 100469, 104547, 108895,   3837,  98316, 103799, 107832, 100265,
          99228,  99849,   1773, 120249,  99546, 101494, 102769, 114272,  48139,
             16,     13,   3070,  99316, 100880,    334,   5122, 101494,  98318,
         100166, 104836,     15, 126930,   3837, 100685, 102071,  99103,  98595,
         100131,   5373,  98630, 100050,   5373, 111574,  98327, 100392,  98485,
         102402,  98401,  98641,   1773,  99281,   3837,  99628, 100039, 102209,
         107408, 124171, 116495,   3407,     17,     13,   3070, 125109,    334,
          28213,    256,    481,  99743,  98611, 101676,   5122, 120069, 111506,
         101676, 102254,   3837,  99002, 116743, 101245, 104667,  99849,   8994,
            256,    481, 107020, 102384, 101676,   5122,  98478, 125833, 118940,
          98314, 1016

In [26]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

history = []

def runChat(user_input:str):
    history.append([user_input, ""])
    messages = []
    
    for idx, (user_msg, model_msg) in enumerate(history):
        if idx == len(history) - 1 and not model_msg:
            messages.append({"role": "user", "content": user_msg})
            break
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if model_msg:
            messages.append({"role": "assistant", "content": model_msg})
    
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)
    
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer,
        timeout=60,
        skip_prompt=True,
        skip_special_tokens=True
    )
    
    generate_kwargs = {
        "input_ids": model_inputs,
        "streamer": streamer,
        "max_new_tokens": 8192,
        "do_sample": True,
        "top_p": 0.8,
        "temperature":  0.6,
        "stopping_criteria": StoppingCriteriaList([StopOnTokens()]),
        "repetition_penalty": 1.2,
        "eos_token_id": model.config.eos_token_id,
    }
    
    result = model.generate(**generate_kwargs)
    print(f"result:{result}")

    generatetxt=""
    for new_token in streamer:
        if new_token:
            print(new_token, end="", flush=True)
            history[-1][1] += new_token
            generatetxt+=new_token
    return generatetxt

user_input = "你好"
res = runChat(user_input)
print(res)
history[-1][1] = history[-1][1].strip()

result:tensor([[151331, 151333, 151336,    198, 109377, 151337,    198, 109377,   9281,
            239,    233,   6313, 118295, 103810,  98406,   3837, 101665, 110368,
          99444,  99212,  11314, 151336]], device='cuda:0')

你好👋！很高兴见到你，有什么可以帮助你的吗？
你好👋！很高兴见到你，有什么可以帮助你的吗？
